# Feature selections: Iterative Method 500 to 20

In [17]:
%run ../__init__.py
%matplotlib inline

In [18]:
target = pd.read_csv('../Datasets/madelon_train.labels', sep=' ', header=None)
train = pd.read_csv('../Datasets/madelon_train.data', sep=' ', header=None)
val = pd.read_csv('../Datasets/madelon_valid.data', sep=' ', header=None)

In [19]:
target.columns = ['target']
train = train.drop(train.columns[500], axis=1)

train= pd.concat([train, target], 1)

X = train.drop(['target'], axis=1)
y = train['target']

In [20]:
sample1 = train.sample(440)

Uci_y_1 = sample1['target']
Uci_X_1 = sample1.drop(['target'], axis=1)
Uci_X_1.shape

(440, 500)

In [5]:
# use this function to find r2 of redundant features 
# dropping a feature and seeing if the other features can predict it
def calculate_r_2_for_feature(data,feature):
    new_data = data.drop(feature, axis=1)

    X_train, \
    X_test,  \
    y_train, \
    y_test = train_test_split(
        new_data,data[feature],test_size=0.25
    )

    regressor = KNeighborsRegressor()
    regressor.fit(X_train,y_train)

    score = regressor.score(X_test,y_test)
    return score

#use this function to take the mean of the scores after 100 runs
def mean_r2_for_feature(data, feature):
    scores = []
    for _ in range(10):
        scores.append(calculate_r_2_for_feature(data, feature))
        
    scores = np.array(scores)
    return scores.mean()

# use this function to get the mean of scores of multiple columns 
def mean_column_range_Knn(data):
    r2_knn = []
    for i in tqdm(range(0,500)):
        if mean_r2_for_feature(data, i) > 0:
            r2_knn.append(i)
    return r2_knn

In [8]:
r2_knn = mean_column_range_Knn(Uci_X_1)

100%|██████████| 500/500 [02:54<00:00,  2.86it/s]


In [9]:
r2_knn

[28,
 48,
 64,
 105,
 128,
 153,
 241,
 281,
 318,
 336,
 338,
 378,
 433,
 442,
 451,
 453,
 455,
 472,
 475,
 493]

In [9]:
# use this function to find r2 of redundant features 
# dropping a feature and seeing if the other features can predict it
def calculate_r_2_for_feature_tree(data,feature):
    new_data = data.drop(feature, axis=1)

    X_train, \
    X_test,  \
    y_train, \
    y_test = train_test_split(
        new_data,data[feature],test_size=0.25
    )

    regressor = DecisionTreeRegressor()
    regressor.fit(X_train,y_train)

    score = regressor.score(X_test,y_test)
    return score

#use this function to take the mean of the scores after 5 runs
def mean_r2_for_feature_tree(data, feature):
    scores = []
    for _ in range(5):
        scores.append(calculate_r_2_for_feature_tree(data, feature))
        
    scores = np.array(scores)
    return scores.mean()

# use this function to get the mean of scores of multiple columns 
def mean_column_range_tree(data):
    r2_tree= []
    for i in tqdm(range(0,500)):
        if mean_r2_for_feature_tree(data, i) > 0:
            r2_tree.append(i)
    return r2_tree

In [10]:
r2_tree = mean_column_range_tree(Uci_X_1)

100%|██████████| 500/500 [04:06<00:00,  2.03it/s]


In [11]:
display(r2_tree)

[28,
 48,
 64,
 105,
 128,
 153,
 241,
 281,
 318,
 336,
 338,
 378,
 433,
 442,
 451,
 453,
 455,
 472,
 475,
 493]

Both top R2 scores from Knn and DecisionTree show the same columns this indicates that these are the top 20 features that are important. So I will keep columns 28, 48, 64, 105, 128, 153, 241, 281, 318, 336, 338, 378, 433, 442, 451, 453, 455, 472, 475, 493)

In [14]:
top_20 = [28, 48, 64, 105, 128, 153, 241, 281, 318, 336, 338, 378, 433,
          442, 451, 453, 455, 472, 475, 493]

In [9]:
def skb_top_feats(x, y):
    X_train, X_test, y_train, y_test = train_test_split(x, 
                                                    y, 
                                                    test_size=.2, 
                                                    random_state=42)
    
    scaler = StandardScaler()
    
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)
    skb_list = []
    skb = SelectKBest(k=20)
    skb.fit(X_train_scaled, y_train)
    
    skb_feats = np.where(skb.get_support())[0]
    
    skb_list.append(skb_feats)
    
    return skb_list

In [10]:
uci_skb_feats = skb_top_feats(Uci_X_1, Uci_y_1)

In [11]:
uci_skb_feats

[array([ 48,  64,  85,  87, 105, 128, 146, 241, 250, 310, 311, 336, 338,
        340, 352, 353, 355, 424, 475, 496])]

In [12]:
uci_skb_feats = [48, 64, 105, 128, 137, 149, 199, 204, 241, 282, 329, 336, 338, 378,
        424, 442, 453, 472, 475, 493]

In [15]:
for i in uci_skb_feats:
    if i in top_20:
        print(i)

48
64
105
128
241
336
338
378
442
453
472
475
493


# Correlation Matrix Method

In [71]:
corr_df = X.corr().abs()
corr_df = corr_df > .5
corr_df = corr_df[corr_df].count()

In [22]:
corr_df = Uci_X_1.corr().abs()
for i in corr_df.columns:
    corr_df.loc[i,i] = 0
corr_list = corr_df.max().sort_values(ascending=False)[:20]

corr_list = pd.DataFrame(corr_list)
corr_list.sort_index(ascending=True)

,0
28,0.989640
48,0.988951
64,0.989398
105,0.990148
128,0.990148
153,0.990329
241,0.990338
281,0.990329
318,0.989640
336,0.989398


Even though SelectKbest did not give the same 20 columns as R2 method, 13 of SelectKbest columns were in R2 method. The correlation matrix method also gave the same top 20 columns as the R2 method.  Therefore, these will be the 20 important columns that would be used to run the models

In [ ]:
train.to_pickle('./Datasets/train_clean.p')